Checking if there is a GPU device available for training. I did the training on Kaggle, so this is just to debug.

In [2]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


CUDA Available: True
Device Name: NVIDIA GeForce RTX 2070 SUPER


In [3]:
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
from d2l import torch as d2l

Below I define a custom Test Dataset class to make the test dataloader

In [4]:

class TestDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_paths = [
            os.path.join(root, fname)
            for fname in os.listdir(root)
            if fname.endswith(('jpg', 'png', 'jpeg'))
        ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_path

def init_cnn(module):
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        nn.init.xavier_uniform_(module.weight)

class NandNet(d2l.Classifier):

    def __init__(self, numchannels, lr=0.1, num_classes=2):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
            nn.LazyConv2d(numchannels, kernel_size=3, stride=1, padding=1),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.LazyConv2d(numchannels, kernel_size=3, stride=1, padding=1),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.LazyConv2d(numchannels, kernel_size=3, stride=1, padding=1),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.Flatten(), nn.LazyLinear(64), nn.LazyBatchNorm1d(),
            nn.ReLU(), nn.LazyLinear(16), nn.LazyBatchNorm1d(),
            nn.ReLU(), nn.LazyLinear(num_classes)
            )

class MyData(d2l.DataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.save_hyperparameters()
        self.train = trainset
        self.val = valset
        self.batch_size = batch_size
        

    def text_labels(self, indices):
        labels = ["yes", "no"]
        return [labels[int(i)] for i in indices]

    def get_dataloader(self, train):
        data = self.train if train else self.val
        return DataLoader(data, self.batch_size, shuffle=train, num_workers=16)

    def visualize(self, batch, nrows=1, ncols=8, labels=None):
        if labels is None:
            labels = []
        X, y = batch
        if not labels:
            labels = self.text_labels(y)
        d2l.show_images(X.squeeze(1), nrows, ncols, titles=labels)


In [5]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(
        size=224,
        scale=(0.85, 1.15),
        ratio=(0.75, 1.33)
    ),
    transforms.ToTensor(),
])

trainset = datasets.ImageFolder(
    root = 'dl2425_challenge_dataset/train',
    transform = transform
)
valset = datasets.ImageFolder(
    root = 'dl2425_challenge_dataset/val',
    transform = transform
)
testset = TestDataset(
    root = 'dl2425_challenge_dataset/test',
    transform = transform
)

testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [6]:
"""if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = NandNet(numchannels=6, lr=0.1, num_classes=2)
    model = model.to(device)
    trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
    dummy_input = torch.randn(64, 3, 224, 224).to(device)
    model(dummy_input)
    model.apply(init_cnn)
    data = MyData(batch_size=64)
    trainer.fit(model, data)
    torch.save(model.state_dict(), 'model_weights.pth')
    model.eval()  # Set to evaluation mode"""

'if __name__ == \'__main__\':\n    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n    model = NandNet(numchannels=6, lr=0.1, num_classes=2)\n    model = model.to(device)\n    trainer = d2l.Trainer(max_epochs=10, num_gpus=1)\n    dummy_input = torch.randn(64, 3, 224, 224).to(device)\n    model(dummy_input)\n    model.apply(init_cnn)\n    data = MyData(batch_size=64)\n    trainer.fit(model, data)\n    torch.save(model.state_dict(), \'model_weights.pth\')\n    model.eval()  # Set to evaluation mode'

In [10]:
# Define the model architecture
print(torch.cuda.is_available())
device = torch.device("cuda")
print(device)
model = NandNet(numchannels=6, lr=0.1, num_classes=2)
model.load_state_dict(torch.load('models/model_weights_3layers_simple.pth'),)
model = model.to(device)
model.eval()

# Test the model
with torch.no_grad():
    for X, _ in testloader:
        X = X.to(device)
        y_pred = model(X)
        probs = torch.softmax(y_pred, dim=1)  # Apply softmax to get probabilities
        class_preds = torch.argmax(probs, dim=1)  # Class with highest probability
        print(f"Predicted Classes: {class_preds}")



True
cuda


C:\Users\Fernando\AppData\Local\Temp\ipykernel_29304\2046883477.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/model_weights_3l

Predicted Classes: tensor([1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1], device='cuda:0')
Predicted Classes: tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], device='cuda:0')
Predicted Classes: tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], device='cuda:0')
Predicted Classes: tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1], devic